# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-25-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-25-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-26 04:21:04,33.93911,67.709953,58843,2582,52489.0,3772.0,Afghanistan,151.157225,4.387948
1,NaN,NaN,NaN,Albania,2021-04-26 04:21:04,41.15330,20.168300,130537,2378,105728.0,22431.0,Albania,4535.999722,1.821706
2,NaN,NaN,NaN,Algeria,2021-04-26 04:21:04,28.03390,1.659600,120922,3207,84299.0,33416.0,Algeria,275.756269,2.652123
3,NaN,NaN,NaN,Andorra,2021-04-26 04:21:04,42.50630,1.521800,13060,124,12491.0,445.0,Andorra,16902.866757,0.949464
4,NaN,NaN,NaN,Angola,2021-04-26 04:21:04,-11.20270,17.873900,25609,579,23092.0,1938.0,Angola,77.918795,2.260924


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2535,2539,2539,2546,2549,2557,2561,2565,2572,2582
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2337,2340,2342,2347,2353,2358,2364,2367,2372,2378
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3148,3152,3155,3160,3165,3172,3181,3190,3198,3207


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,52105,52116,52168,52244,52272,52301,52348,52363,52392,52489
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,100600,101142,101584,102171,102601,103066,103582,104278,105016,105728
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,83169,83286,83397,83514,83636,83765,83900,84038,84167,84299


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6750,6760,6763,6763,6773,6793,6819,6835,6876,6879
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20701,20714,20723,20730,20764,20787,20815,20833,20838,20847
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2264,2271,2271,2271,2275,2284,2289,2292,2296,2296


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,103,106,106,106,106,107,107,107,107,107
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,302,302,302,302,302,302,303,303,305,305
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,55,55,55,55,55,55,56,56,56,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-26 04:21:04,32.539527,-86.644082,6879,107,NaN,NaN,"Autauga, Alabama, US",12312.731568,1.555459
689,1075.0,Lamar,Alabama,US,2021-04-26 04:21:04,33.779950,-88.096680,1404,34,NaN,NaN,"Lamar, Alabama, US",10170.228178,2.421652
690,1077.0,Lauderdale,Alabama,US,2021-04-26 04:21:04,34.901719,-87.656247,9322,236,NaN,NaN,"Lauderdale, Alabama, US",10052.949994,2.531646


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32077075,572200,0.0
India,16960172,192311,14085110.0
Brazil,14340787,390797,12614559.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32077075,572200,0.0,31504875.0,2021-04-26 04:21:04,37.950547,-91.419547
India,16960172,192311,14085110.0,2682751.0,2021-04-26 04:21:04,23.088275,81.806127
Brazil,14340787,390797,12614559.0,1335431.0,2021-04-26 04:21:04,-12.669522,-48.480493
France,5559121,103017,338191.0,5117913.0,2021-04-26 04:21:04,6.430808,-34.730285
Russia,4708640,106434,4338114.0,264092.0,2021-04-26 04:21:04,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3731667,61479,0.0
Texas,2873995,49936,0.0
Florida,2208584,34848,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3731667,61479,0.0,3670188.0,2021-04-26 04:21:04,37.843962,-120.728594
Texas,2873995,49936,0.0,2824059.0,2021-04-26 04:21:04,31.660643,-98.653069
Florida,2208584,34848,0.0,2173736.0,2021-04-26 04:21:04,28.940755,-82.700744
New York,2031093,52003,0.0,1979090.0,2021-04-26 04:21:04,42.544151,-75.474183
Illinois,1320934,24139,0.0,1296795.0,2021-04-26 04:21:04,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1231532,23802,0.0
Arizona,Maricopa,534430,9836,0.0
Illinois,Cook,529165,9894,0.0
Florida,Miami-Dade,478660,6112,0.0
Texas,Harris,390174,6150,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1231532,23802,0.0,1207730.0,2021-04-26 04:21:04,34.308284,-118.228241,6037.0
Arizona,Maricopa,534430,9836,0.0,524594.0,2021-04-26 04:21:04,33.348359,-112.491815,4013.0
Illinois,Cook,529165,9894,0.0,519271.0,2021-04-26 04:21:04,41.841448,-87.816588,17031.0
Florida,Miami-Dade,478660,6112,0.0,472548.0,2021-04-26 04:21:04,25.611236,-80.551706,12086.0
Texas,Harris,390174,6150,0.0,384024.0,2021-04-26 04:21:04,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,58542,130270,120562,13007,25279,1222,2824652,212114,29653,604823,...,4416588,179537,88280,4,189381,2830,289120,6056,91251,38045
2021-04-24,58730,130409,120736,13024,25492,1227,2845872,212878,29663,606954,...,4418710,182326,88679,4,190523,2833,290259,6105,91317,38064
2021-04-25,58843,130537,120922,13060,25609,1227,2860884,213288,29680,608979,...,4420443,184865,89027,4,191518,2843,290972,6137,91358,38086


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,2565,2367,3190,123,574,31,61176,3984,910,10055,...,127638,2227,641,1,2009,35,3138,1165,1245,1556
2021-04-24,2572,2372,3198,124,577,31,61474,4001,910,10070,...,127670,2283,642,1,2028,35,3151,1175,1245,1556
2021-04-25,2582,2378,3207,124,579,31,61644,4018,910,10087,...,127681,2326,643,1,2047,35,3167,1187,1246,1557


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,52363,104278,84038,12423,23089,995,2474514,192281,23191,568213,...,14281,148561,85232,3,172035,2490,259105,2454,89256,35094
2021-04-24,52392,105016,84167,12458,23092,1002,2496277,193360,23195,570684,...,14388,151669,85536,3,173097,2490,260650,2486,89403,35101
2021-04-25,52489,105728,84299,12491,23092,1002,2518167,194080,23209,573155,...,14452,154489,85764,3,173097,2516,262602,2546,89574,35123


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6750,6760,6763,6763,6773,6793,6819,6835,6876,6879
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1397,1397,1399,1400,1400,1398,1398,1399,1403,1404
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9265,9271,9276,9280,9286,9293,9301,9310,9317,9322
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2959,2962,2964,2963,2969,2976,2977,2982,2982,2983
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15589,15614,15631,15638,15642,15670,15694,15709,15722,15723


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,525477,67059,0,857347,334458,3729116,498758,334766,102593,49,...,841049,2870407,394678,22416,3068,652321,393594,150693,655410,57696
2021-04-24,525898,67059,0,858076,334642,3731145,500675,334766,102927,49,...,841049,2872756,395150,22479,3068,653326,395312,151068,656061,57696
2021-04-25,526131,67059,0,858737,334706,3731667,502242,334766,103222,49,...,841049,2873995,395431,22576,3068,654210,395312,151382,656526,57696


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-23        6835   20833    2292  2578   6570    1222   2146   14399   
2021-04-24        6876   20838    2296  2582   6570    1223   2146   14400   
2021-04-25        6879   20847    2296  2584   6571    1224   2146   14405   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-23         3529     1832  ...    2670    619     3112      765   
2021-04-24         3529     1833  ...    2670    619     3112      765   
2021-04-25         3532     1833  ...    2670    619     3112      765   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-23           4159  3719  2158          0      898    648  
2021-04-24           4159  3719  2158          0      898    648  
2021-04-25           4159  3719  2158          0      898    648  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,10840,334,0,17238,5716,61401,6231,8047,1609,0,...,12142,49877,2179,243,27,10666,5434,2813,7452,705
2021-04-24,10851,334,0,17260,5717,61445,6249,8047,1615,0,...,12142,49921,2180,244,27,10675,5434,2817,7474,705
2021-04-25,10851,334,0,17268,5718,61479,6251,8047,1616,0,...,12142,49936,2182,244,27,10691,5434,2821,7473,705


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-23         107     303      56   62    134      40     68     313   
2021-04-24         107     305      56   62    134      40     69     312   
2021-04-25         107     305      56   62    134      40     69     312   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-23          121       45  ...      30     11       31        7   
2021-04-24          121       45  ...      30     11       31        7   
2021-04-25          121       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-23             37     9    12          0       26      5  
2021-04-24             37     9    12          0       26      5  
2021-04-25             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.003944,0.001199,0.001653,0.005022,0.009101,0.004108,0.009970,0.003382,0.000506,0.003866,...,0.000624,0.020729,0.003923,0.0,0.007008,0.002125,0.005006,0.005980,0.000680,0.000710
2021-04-24,0.003211,0.001067,0.001443,0.001307,0.008426,0.004092,0.007512,0.003602,0.000337,0.003523,...,0.000480,0.015534,0.004520,0.0,0.006030,0.001060,0.003940,0.008091,0.000723,0.000499
2021-04-25,0.001924,0.000982,0.001541,0.002764,0.004590,0.000000,0.005275,0.001926,0.000573,0.003336,...,0.000392,0.013926,0.003924,0.0,0.005222,0.003530,0.002456,0.005242,0.000449,0.000578


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.001562,0.001269,0.002829,0.00000,0.003497,0.0,0.009172,0.003779,0.0,0.002892,...,0.000321,0.031019,0.001562,0.0,0.011072,0.0,0.007384,0.006914,0.004032,0.000643
2021-04-24,0.002729,0.002112,0.002508,0.00813,0.005226,0.0,0.004871,0.004267,0.0,0.001492,...,0.000251,0.025146,0.001560,0.0,0.009457,0.0,0.004143,0.008584,0.000000,0.000000
2021-04-25,0.003888,0.002530,0.002814,0.00000,0.003466,0.0,0.002765,0.004249,0.0,0.001688,...,0.000086,0.018835,0.001558,0.0,0.009369,0.0,0.005078,0.010213,0.000803,0.000643


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.000287,0.006719,0.001645,0.003879,0.008209,0.000000,0.010091,0.005875,0.000345,0.004774,...,0.006058,0.021290,0.002647,2.0,0.005800,0.000000,0.009924,0.025491,0.001560,0.000599
2021-04-24,0.000554,0.007077,0.001535,0.002817,0.000130,0.007035,0.008795,0.005612,0.000172,0.004349,...,0.007492,0.020921,0.003567,0.0,0.006173,0.000000,0.005963,0.013040,0.001647,0.000199
2021-04-25,0.001851,0.006780,0.001568,0.002649,0.000000,0.000000,0.008769,0.003724,0.000604,0.004330,...,0.004448,0.018593,0.002666,0.0,0.000000,0.010442,0.007489,0.024135,0.001913,0.000627


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,0.000815,0.002242,NaN,0.001046,0.000706,0.000323,0.003776,0.003098,0.003188,0.0,...,0.001438,0.000927,0.000872,0.004076,0.0,0.002058,0.004479,0.002695,0.001375,0.001441
2021-04-24,0.000801,0.000000,NaN,0.000850,0.000550,0.000544,0.003844,0.000000,0.003256,0.0,...,0.000000,0.000818,0.001196,0.002810,0.0,0.001541,0.004365,0.002489,0.000993,0.000000
2021-04-25,0.000443,0.000000,NaN,0.000770,0.000191,0.000140,0.003130,0.000000,0.002866,0.0,...,0.000000,0.000431,0.000711,0.004315,0.0,0.001353,0.000000,0.002079,0.000709,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-23      0.002346  0.000865  0.001311  0.001943  0.001067 -0.000818   
2021-04-24      0.005999  0.000240  0.001745  0.001552  0.000000  0.000818   
2021-04-25      0.000436  0.000432  0.000000  0.000775  0.000152  0.000818   

Province_State                                        ...  Wyoming            \
Admin2          Butler   Calhoun  Chambers  Cherokee  ...     Park    Platte   
2021-04-23      0.0014  0.000556  0.000567  0.000000  ...  0.00075  0.001618   
2021-04-24      0.0000  0.000069  0.000000  0.000546  ...  0.00000  0.000000   
2021-04-25      0.0000  0.000347  0.000850  0.000000  ...  0.00000  0.000000   

Province_State                                                               \
Admin2         Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-04-23          0.0  0.010568   0.001927  0.001077  0.000464        NaN   
2021-04-24          0.0  0.000000   0.000000  0.000000  0.000000        NaN   
2021-04-25          0.0  0.000000   0.000000  0.000000  0.000000        NaN   

Province_State                     
Admin2         Washakie    Weston  
2021-04-23          0.0  0.001546  
2021-04-24          0.0  0.000000  
2021-04-25          0.0  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,0.001478,0.0,NaN,0.000987,0.000876,0.001501,0.000964,0.000995,0.003117,NaN,...,0.000989,0.001184,0.000459,0.000000,0.0,0.001220,0.001105,0.001781,0.001882,0.0
2021-04-24,0.001015,0.0,NaN,0.001276,0.000175,0.000717,0.002889,0.000000,0.003729,NaN,...,0.000000,0.000882,0.000459,0.004115,0.0,0.000844,0.000000,0.001422,0.002952,0.0
2021-04-25,0.000000,0.0,NaN,0.000463,0.000175,0.000553,0.000320,0.000000,0.000619,NaN,...,0.000000,0.000300,0.000917,0.000000,0.0,0.001499,0.000000,0.001420,-0.000134,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga   Baldwin Barbour Bibb    Blount Bullock    Butler   
2021-04-23         0.0  0.000000     0.0  0.0  0.007519     0.0  0.000000   
2021-04-24         0.0  0.006601     0.0  0.0  0.000000     0.0  0.014706   
2021-04-25         0.0  0.000000     0.0  0.0  0.000000     0.0  0.000000   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-04-23      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2021-04-24     -0.003195      0.0      0.0  ...     0.0    0.0      0.0   
2021-04-25      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-23          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-24          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-25          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.003026,0.001122,0.001592,0.003854,0.008057,0.002127,0.009764,0.003708,0.000784,0.003902,...,0.000611,0.019373,0.003922,0.041667,0.006794,0.003029,0.005483,0.007563,0.000751,0.001140
2021-04-24,0.003119,0.001095,0.001518,0.002581,0.008241,0.003110,0.008638,0.003655,0.000561,0.003713,...,0.000546,0.017454,0.004221,0.020833,0.006412,0.002045,0.004711,0.007827,0.000737,0.000820
2021-04-25,0.002521,0.001038,0.001529,0.002672,0.006415,0.001555,0.006957,0.002791,0.000567,0.003525,...,0.000469,0.015690,0.004073,0.010417,0.005817,0.002787,0.003584,0.006534,0.000593,0.000699


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.001742,0.001797,0.002583,0.000067,0.004107,0.000017,0.007952,0.004960,2.685815e-07,0.003048,...,0.000243,0.032247,0.001142,0.0,0.010923,2.881175e-72,0.007204,0.007382,0.002679,0.000467
2021-04-24,0.002236,0.001955,0.002545,0.004098,0.004667,0.000009,0.006412,0.004614,1.342908e-07,0.002270,...,0.000247,0.028697,0.001351,0.0,0.010190,1.440588e-72,0.005673,0.007983,0.001339,0.000233
2021-04-25,0.003062,0.002242,0.002680,0.002049,0.004066,0.000004,0.004589,0.004431,6.714539e-08,0.001979,...,0.000166,0.023766,0.001454,0.0,0.009779,7.202938e-73,0.005375,0.009098,0.001071,0.000438


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-23,0.000538,0.005777,0.001596,0.003827,0.005901,0.000605,0.009639,0.005069,0.000256,0.004616,...,0.005352,0.021560,0.002790,1.00,0.006867,0.000476,0.009628,0.024735,0.001441,0.000456
2021-04-24,0.000546,0.006427,0.001566,0.003322,0.003015,0.003820,0.009217,0.005340,0.000214,0.004483,...,0.006422,0.021240,0.003178,0.50,0.006520,0.000238,0.007795,0.018888,0.001544,0.000328
2021-04-25,0.001199,0.006604,0.001567,0.002986,0.001508,0.001910,0.008993,0.004532,0.000409,0.004406,...,0.005435,0.019917,0.002922,0.25,0.003260,0.005340,0.007642,0.021511,0.001728,0.000477


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,0.001033,0.002696,NaN,0.000907,0.000661,0.000488,0.003855,0.002812,0.002960,1.647911e-122,...,0.001777,0.001134,0.001016,0.004045,0.002376,0.002037,0.004232,0.002643,0.001305,0.001423
2021-04-24,0.000917,0.001348,NaN,0.000879,0.000606,0.000516,0.003849,0.001406,0.003108,8.239557e-123,...,0.000888,0.000976,0.001106,0.003428,0.001188,0.001789,0.004299,0.002566,0.001149,0.000711
2021-04-25,0.000680,0.000674,NaN,0.000825,0.000398,0.000328,0.003489,0.000703,0.002987,4.119779e-123,...,0.000444,0.000704,0.000909,0.003871,0.000594,0.001571,0.002149,0.002322,0.000929,0.000356


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-23      0.002615  0.001039  0.001842  0.001467  0.001134  0.000090   
2021-04-24      0.004307  0.000640  0.001794  0.001509  0.000567  0.000454   
2021-04-25      0.002372  0.000536  0.000897  0.001142  0.000360  0.000636   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-23      0.001362  0.000697  0.000560  0.000557  ...  0.000592   
2021-04-24      0.000681  0.000383  0.000280  0.000551  ...  0.000296   
2021-04-25      0.000341  0.000365  0.000565  0.000276  ...  0.000148   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-23      0.000922  0.000347  0.005735   0.001853  0.001098  0.000599   
2021-04-24      0.000461  0.000173  0.002867   0.000926  0.000549  0.000299   
2021-04-25      0.000230  0.000087  0.001434   0.000463  0.000275  0.000150   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2021-04-23           -1.0  7.003103e-07  0.000828  
2021-04-24           -1.0  3.501552e-07  0.000414  
2021-04-25           -1.0  1.750776e-07  0.000207  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-23,0.001324,0.000125,NaN,0.001013,0.000730,0.001486,0.001115,0.001108,0.001795,NaN,...,0.001208,0.001247,0.000782,0.000593,0.001202,0.001360,0.001428,0.002246,0.001328,0.000179
2021-04-24,0.001169,0.000063,NaN,0.001145,0.000452,0.001101,0.002002,0.000554,0.002762,NaN,...,0.000604,0.001065,0.000620,0.002354,0.000601,0.001102,0.000714,0.001834,0.002140,0.000090
2021-04-25,0.000585,0.000031,NaN,0.000804,0.000314,0.000827,0.001161,0.000277,0.001691,NaN,...,0.000302,0.000683,0.000769,0.001177,0.000300,0.001300,0.000357,0.001627,0.001003,0.000045


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2021-04-23      0.001407  0.000841  0.004545  0.004372  0.003687   
2021-04-24      0.000703  0.003721  0.002273  0.002186  0.001844   
2021-04-25      0.000352  0.001860  0.001136  0.001093  0.000922   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2021-04-23      1.528326e-09  0.000131  0.000804 -0.002049  0.001422  ...   
2021-04-24      7.641629e-10  0.007419 -0.001195 -0.001025  0.000711  ...   
2021-04-25      3.820814e-10  0.003709 -0.000598 -0.000512  0.000355  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-04-23      8.029700e-12  3.259225e-28  6.064532e-14  2.664203e-34   
2021-04-24      4.014850e-12  1.629612e-28  3.032266e-14  1.332102e-34   
2021-04-25      2.007425e-12  8.148061e-29  1.516133e-14  6.660508e-35   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-04-23      1.655685e-09  8.504638e-22  4.322005e-29  -0.501961   
2021-04-24      8.278423e-10  4.252319e-22  2.161002e-29  -0.501961   
2021-04-25      4.139211e-10  2.126159e-22  1.080501e-29  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-23      1.655314e-26  3.515518e-25  
2021-04-24      8.276572e-27  1.757759e-25  
2021-04-25      4.138286e-27  8.788794e-26  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
